In [2]:
### Declare variables, import packages
from random import *
from math import *
import csv
import numpy as np
from munkres import Munkres, print_matrix
generate = False
with open('cost_function.txt') as f:
    content = f.readlines()
def cost_function(L, F): 
    return eval(content[0])

In [3]:
### Get Cost Matrix
def random_ranking(numrow,numcol):
    mat = [["O" for a in range(numcol)] for b in range(numrow)]  # final combined preference matrix [numrow x numcol]
    for row in range(numrow):
        temp = []
        for i in range(1, numcol + 1):
            temp.append(i)
        shuffle(temp)
        mat[row] = temp
    return np.array(mat)

def makelist(prefix, n):
    values = []
    for i in range(n):
        values.append("%s%d" % (prefix,i))
    return values

if (generate):
    # Create a random set of preference for each leader, append into matrix1
    seed(12345)
    num_leaders = 8
    num_follow = 12
    matrix1 = random_ranking(num_leaders,num_follow)
    matrix2 = random_ranking(num_follow,num_leaders)
    leaderlist = dict(zip(range(numrow), makelist("L",numrow)))
    followerlist = dict(zip(range(numcol), makelist("F",numcol)))
    numrow = num_leaders
    numcol = num_follow
else:
    L_input = csv.reader(open("leader_matrix.csv","rb"),delimiter=',')
    L_header = next(L_input)
    L_header = [L_header[i] for i in range(1,len(L_header))]
    matrix1 = np.array(list(L_input))[:, 1:].astype(np.integer)
    F_input = csv.reader(open("follower_matrix.csv","rb"),delimiter=',')
    F_header = next(F_input)
    F_header = [F_header[i] for i in range(1,len(F_header))]
    matrix2 = np.array(list(F_input))[:, 1:].astype(np.integer)
    numrow = matrix1.shape[0]
    numcol = matrix1.shape[1]
    leaderlist = dict(zip(range(numrow), F_header))
    followerlist = dict(zip(range(numcol), L_header))

matrix = cost_function(matrix1,matrix2.T)
matrix_original = matrix.copy()

In [4]:
# Print Function
def printmatrix(matrix, dict1, dict2, sides): 
    left_bottom = np.array(np.c_[dict1.values(), matrix])
    if (sides):
        top = np.array([''] + dict2.values())[np.newaxis]
        matrix = np.r_[top, left_bottom]
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print '\n'.join(table)

print("\nPREFERENCE MATRIX FOR LEADERS (LEADERS LEFT X FOLLOWERS TOP)")
printmatrix(matrix1, leaderlist, followerlist, False)
print("\nPREFERENCE MATRIX FOR FOLLOWERS (FOLLOWERS LEFT X LEADERS TOP)")
printmatrix(matrix2, followerlist, leaderlist, False)
print("\nCOMBINED PREFERENCE MATRIX (LEADERS LEFT X FOLLOWERS TOP)")
printmatrix(matrix, leaderlist, followerlist, False)


PREFERENCE MATRIX FOR LEADERS (LEADERS LEFT X FOLLOWERS TOP)
2	4 	10	6 	9 	8	3	7 	1	5 
8	7 	9 	1 	10	3	4	2 	6	5 
5	4 	8 	7 	1 	3	9	10	2	6 
4	3 	2 	6 	9 	5	7	10	1	8 
3	8 	1 	9 	10	5	4	2 	7	6 
7	9 	1 	6 	10	4	2	5 	8	3 
3	6 	5 	7 	1 	4	9	2 	8	10
7	10	4 	3 	2 	5	6	8 	9	1 
4	2 	9 	10	7 	8	5	3 	6	1 
5	3 	4 	10	9 	8	7	2 	1	6 

PREFERENCE MATRIX FOR FOLLOWERS (FOLLOWERS LEFT X LEADERS TOP)
3 	3 	10	1 	2 	4 	8 	8 	4 	7 
5 	5 	4 	5 	6 	9 	2 	7 	3 	2 
7 	10	6 	4 	1 	7 	9 	1 	7 	4 
9 	7 	1 	3 	5 	2 	6 	5 	6 	5 
6 	1 	8 	6 	7 	10	4 	9 	10	10
8 	6 	9 	8 	10	3 	5 	6 	5 	1 
2 	2 	3 	10	9 	5 	7 	3 	9 	9 
4 	4 	2 	9 	4 	6 	10	10	2 	3 
10	9 	5 	2 	3 	1 	1 	4 	1 	8 
1 	8 	7 	7 	8 	8 	3 	2 	8 	6 

COMBINED PREFERENCE MATRIX (LEADERS LEFT X FOLLOWERS TOP)
13	23	53	45	48 	40	14	38	104	33
55	35	48	53	113	30	20	18	45 	40
60	20	40	59	68 	63	69	98	25 	33
23	23	18	33	48 	40	50	49	8  	39
14	40	5 	53	53 	60	55	18	43 	38
38	45	53	38	50 	19	25	28	75 	50
50	38	49	34	20 	23	45	90	75 	85
38	53	23	23	73 	28	33	48	60 	8 

In [5]:
### Leader Scores
print("\nLEADER SCORES")
leader_scores = np.mean(matrix2, axis=1)
for i in np.argsort(leader_scores):
    print '%s: %s' % (leaderlist.values()[i], round(leader_scores[i],2))

### Follower Scores
print("\nFOLLOWER SCORES")
follow_scores = np.mean(matrix1, axis=0)
for i in np.argsort(follow_scores):
    print '%s: %s' % (followerlist.values()[i], round(follow_scores[i],2))


LEADER SCORES
Leader_9: 4.4
Leader_2: 4.8
Leader_4: 4.9
Leader_1: 5.0
Leader_8: 5.4
Leader_3: 5.6
Leader_10: 5.8
Leader_7: 5.9
Leader_6: 6.1
Leader_5: 7.1

FOLLOWER SCORES
Follower_1: 4.8
Follower_9: 4.9
Follower_8: 5.1
Follower_10: 5.1
Follower_3: 5.3
Follower_6: 5.3
Follower_2: 5.6
Follower_7: 5.6
Follower_4: 6.5
Follower_5: 6.8


In [11]:
m = Munkres()
indexes = m.compute(matrix)
print("\nFINAL PARTNER ASSIGNMENTS: (N[X] means Person N was given their Xth choice)")
print("------------------------------------------------------------")
csv_write = csv.writer(open('match_outputs.csv','wb'),delimiter = ',',quoting=csv.QUOTE_MINIMAL)
csv_write.writerow(['Leader_Name','Choice_Received','Follower_Name','Choice_Received','Penalty'])
total = 0
for row, col in indexes:
    value = matrix_original[row][col]
    total += value
    csv_write.writerow([leaderlist[row], matrix1[row,col], followerlist[col], matrix2[col, row], value])
    print '%s[%d] + %s[%d] >>> %d' % (leaderlist[row], matrix1[row,col], followerlist[col], matrix2[col, row], value)
print 'Total Penalty: %d' % total
csv_write.writerow(['\nTotal Penalty\n', total])


FINAL PARTNER ASSIGNMENTS: (N[X] means Person N was given their Xth choice)
------------------------------------------------------------
Leader_1[2] + Follower_1[3] >>> 13
Leader_2[4] + Follower_7[2] >>> 20
Leader_3[6] + Follower_10[7] >>> 33
Leader_4[1] + Follower_9[2] >>> 8
Leader_5[1] + Follower_3[1] >>> 5
Leader_6[4] + Follower_6[3] >>> 19
Leader_7[1] + Follower_5[4] >>> 20
Leader_8[3] + Follower_4[5] >>> 23
Leader_9[2] + Follower_2[3] >>> 13
Leader_10[2] + Follower_8[3] >>> 13
Total Penalty: 167


In [188]:
# STABLE MATCH ALGORITHM: FAVORS LEADERS
if numrow == numcol:
    print("STABLE MATCH ALGORITHM:")
    singlelist = [0 for a in range(numrow)]  # 0=single
    choicelist = [numrow + 1 for a in range(numcol)]  # all high numbers
    tempmatrix = matrix1.copy()
    print('Starting State')
    print(singlelist)
    print(choicelist)
    print
    counter = 1
    while sorted(singlelist)[0] == 0:  # main algorithm loop, per round
        print("\nROUND #%s" % counter)
        for n in range(len(tempmatrix)):  # For each suitor
            if singlelist[n] == 0:  # If they're single
                p = np.argmin(tempmatrix[n])#tempmatrix[n].index(min(tempmatrix[n]))  # col identifying their first choice
                print("Suitor %s Proposes to Lady %s (His #%s choice)" % (n + 1, p + 1, min(tempmatrix[n])))
                tempmatrix[n][p] = 99  # removes from consideration for next round
                if choicelist[p] > matrix2[p][
                    n]:  # hate of former suitor > hate of new suitor i.e. if you want to trade
                    if choicelist[p] != numrow + 1:
                        print("TRADE UP - Lady drops Suitor %s" % (1 + matrix2[p].tolist().index(choicelist[p])))
                        singlelist[matrix2[p].tolist().index(choicelist[p])] = 0  # set jinked suitor to single
                    singlelist[n] = min(tempmatrix[n]) - 1  # set new suitor to engaged
                    choicelist[p] = matrix2[p][n]  # completes replacement
                else:
                    print("REJECTION")
                print(singlelist)
                print(choicelist)
                print
        counter += 1
    setsum = 0
    print("STABLE MATCH COMPLETE")
    print("N[X] means Person N was given their Xth choice")
    print("------------------------------------------------------------")
    for row in leaderlist:
        followid = matrix1[row].tolist().index(singlelist[row])
        print('%s[%s] + %s[%s] >>> %s' % (leaderlist[row], singlelist[row], followerlist[followid], 
                                        matrix2[followid,row], matrix_original[row,followid]))
        setsum += matrix_original[row,followid]
    print("PENALTY: " + str(setsum))
else:
    print("SKIPPED: STABLE MATCH REQUIRES SQUARE COST MATRIX")

STABLE MATCH ALGORITHM:
Starting State
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[11, 11, 11, 11, 11, 11, 11, 11, 11, 11]


ROUND #1
Suitor 1 Proposes to Lady 9 (His #1 choice)
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[11, 11, 11, 11, 11, 11, 11, 11, 10, 11]

Suitor 2 Proposes to Lady 4 (His #1 choice)
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[11, 11, 11, 7, 11, 11, 11, 11, 10, 11]

Suitor 3 Proposes to Lady 5 (His #1 choice)
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[11, 11, 11, 7, 8, 11, 11, 11, 10, 11]

Suitor 4 Proposes to Lady 9 (His #1 choice)
TRADE UP - Lady drops Suitor 1
[0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[11, 11, 11, 7, 8, 11, 11, 11, 2, 11]

Suitor 5 Proposes to Lady 3 (His #1 choice)
[0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[11, 11, 1, 7, 8, 11, 11, 11, 2, 11]

Suitor 6 Proposes to Lady 3 (His #1 choice)
REJECTION
[0, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[11, 11, 1, 7, 8, 11, 11, 11, 2, 11]

Suitor 7 Proposes to Lady 5 (His #1 choice)
TRADE UP - Lady drops Suitor 3
[0, 1, 0, 1, 1, 0, 1, 0, 0, 0]
[11, 11, 1, 7, 4, 11, 11, 11, 2, 11]

Suito